In [12]:
# This file makes several different dictionaries with vertices and edges so you can get vectors 
# for smaller periods of time. This also allows you to have the day of the year as the date.

# Change search terms with the other variables, but also make sure to change the column
# names for the export process to include the search words.

In [13]:
import csv
import pandas as pd
import json
from json import JSONEncoder
from pandas import DataFrame
from pytrends.request import TrendReq
import numpy as np

# Login to Google. Only need to run this once, the rest of requests will use the same session.
pytrend = TrendReq(hl='en-US', tz=360)

In [14]:
# List of search terms
keyword = (['coronavirus', 'covid'])          #ADD SEARCH TERMS HERE
location = ('US-UT')
timeframe = ('2020-01-01 2020-01-31',
             '2020-02-01 2020-02-28',
             '2020-03-01 2020-03-31',
             '2020-04-01 2020-04-30', 
             '2020-05-01 2020-05-31', 
             '2020-06-01 2020-06-30',
             '2020-07-01 2020-07-31',
             '2020-08-01 2020-08-31',
             '2020-09-01 2020-09-30', 
             '2020-10-01 2020-10-31',
             '2020-11-01 2020-11-30',
             '2020-12-01 2020-12-25')

filename = ('multiplevector.txt')

In [10]:
# Creates separate json formats for each month so you can produce a vector for each month:
#################################################################
for time in timeframe: 
      # Create payload and capture API tokens 
    pytrend.build_payload(keyword, cat=0, timeframe=time,geo=location,gprop='')
      # Create dataframe of interest over time
    df = pytrend.interest_over_time()
      #Format dataframe:
    df = df.reset_index() 
    new_date_col = df['date'].dt.dayofyear
    df = df.drop(columns = ['date', 'isPartial'])
    df.insert(loc=0, column='Date', value=new_date_col)
    df.columns=['Date','Search: coronavirus','Search: covid']        #ADD THE SEARCH TERMS HERE
    df['Location'] = location
#     Define vertices and edges:
    df_list = df.values.tolist()
    vertices = { i : df_list[i] for i in range(0, len(df_list)) }
    edges = []
    for i in range(1,len(df)):
        edge = [str(i-1),str(i)]
        edge = edges.append(edge)
    finaldict = {"Data": { "Vertices": vertices, "Edges": edges}}
    with open(filename, "a") as outfile:
         json.dump(finaldict, outfile)

In [11]:
df

,Date,Search: coronavirus,Search: covid,Location
0,336,20,81,US-UT
1,337,22,90,US-UT
2,338,18,93,US-UT
3,339,22,77,US-UT
4,340,17,73,US-UT
5,341,20,68,US-UT
6,342,22,92,US-UT
7,343,17,80,US-UT
8,344,20,86,US-UT
9,345,23,100,US-UT


In [ ]:
#EXPORTING TO DATABASE:
# Make sure the database has the new columns for these keywords. 
##########################################################
import sqlite3
from pandas import DataFrame

conn = sqlite3.connect('DATABASE.db')
c = conn.cursor()
conn.commit()

df.to_sql("Search_Data",conn,if_exists='append', index = False)
conn.close()

In [ ]:
# EXPORTING TO CSV FILE
##########################################################
df.to_csv('year_search_data.csv')